# TAREA DE EJERCICIO

Actualiza el proyecto del día 1 para resumir una página web y utilizar un modelo de código abierto que se ejecute localmente a través de Ollama en lugar de OpenAI

Podrás utilizar esta técnica para todos los proyectos posteriores si prefiere no utilizar API de pago (closed source).

**Beneficios:**
1. Sin cargos por API: código abierto
2. Los datos no salen de su ordenador

**Desventajas:**
1. Tiene mucha menos potencia (parámetros) que el modelo Frontier

## Resumen de la instalación de Ollama

¡Simplemente visita [ollama.com](https://ollama.com) e instálalo!

Una vez que hayas terminado, el servidor ollama ya debería estar ejecutándose localmente.
Si entras en:
[http://localhost:11434/](http://localhost:11434/)

Debería ver el mensaje `Ollama se está ejecutando`.

De lo contrario, abre una nueva Terminal (Mac) o Powershell (Windows) e introduce `ollama serve`.
Luego, intenta entrar em [http://localhost:11434/](http://localhost:11434/) nuevamente.

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [2]:
# Constantes

OLLAMA_API = "http://127.0.0.1:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "gpt-oss:20b"

##ollama serve
##461  ollama run gpt-oss:20b

In [3]:
# Crea una lista de mensajes utilizando el mismo formato que usamos para OpenAI

messages = [
    {"role": "user", "content": "Describe algunas de las aplicaciones comerciales de la IA generativa."}
]

In [4]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [5]:
response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

**Aplicaciones comerciales de la IA generativa**

La IA generativa—la capacidad de crear contenido nuevo a partir de datos de entrenamiento—ha revolucionado numerosos sectores. A continuación se describen algunas de las aplicaciones comerciales más relevantes, con ejemplos y el valor que aportan.

| Sector | Aplicación | Ejemplo concreto | Beneficio clave |
|--------|------------|------------------|-----------------|
| **Marketing y publicidad** | Generación de copys, slogans y anuncios | *Copy.ai*, *Jasper*, *Writesonic* crean textos persuasivos en segundos | Reducción de tiempos de producción, pruebas A/B más rápidas y contenidos personalizados a escala. |
| **Diseño gráfico y creativo** | Creación de logos, mockups y ilustraciones | *Canva* integra IA generativa para proponer diseños automáticamente; *RunwayML* permite generar imágenes a partir de prompts | Acelera el flujo de trabajo creativo, reduce costos de contratación y permite iterar sobre ideas sin bloqueos. |
| **Desarrollo

# Presentación del paquete ollama

Ahora haremos lo mismo, pero utilizando el elegante paquete de Python ollama en lugar de una llamada HTTP directa.

En esencia, se realiza la misma llamada que se indicó anteriormente al servidor ollama que se ejecuta en localhost:11434

In [6]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

### Aplicaciones comerciales de la IA generativa

La IA generativa, es decir, modelos que pueden **crear** texto, imágenes, audio, código o datos sintéticos, se ha convertido en un activo estratégico para muchas industrias. A continuación se describen las aplicaciones comerciales más relevantes, agrupadas por sector y tipo de valor que aportan.

| Sector | Tipo de generador | Ejemplo de aplicación | Beneficio principal |
|--------|-------------------|-----------------------|---------------------|
| **Marketing & Publicidad** | Texto (ChatGPT, Claude, Gemini) | Copy de anuncios, newsletters, scripts de video, respuestas a preguntas de clientes | Acelera la producción de contenido, reduce costos y mejora la personalización. |
| | Imágenes (Stable Diffusion, Midjourney, DALL‑E) | Arte publicitario, mockups de productos, contenido visual para redes sociales | Genera imágenes a demanda, sin necesidad de fotografía profesional. |
| **E‑commerce** | Producto + descripción | Descripciones auto

# AHORA el ejercicio para ti

Toma el código del día 1 e incorpóralo aquí para crear un resumidor de sitios web que use gpt-oss:20b ejecutándose localmente en lugar de OpenAI

In [9]:
import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
import ollama

## API LOCAL OLLAMA con ollama run gpt-oss:20b
OLLAMA_API = "http://127.0.0.1:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "gpt-oss:20b"


## Clase 
# Una clase para representar una página web
# Si no estás familiarizado con las clases, consulta el cuaderno "Python intermedio"

class Website:
    """
    Una clase de utilidad para representar un sitio web que hemos scrappeado
    """

    def __init__(self, url):
        """
        Crea este objeto de sitio web a partir de la URL indicada utilizando la biblioteca BeautifulSoup
        """
        self.url = url
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No tiene título"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)


# Una función que escribe un mensaje de usuario que solicita resúmenes de sitios web:

def user_prompt_for(website):
    user_prompt = f"Estás viendo un sitio web titulado {website.title}"
    user_prompt += "\nEl contenido de este sitio web es el siguiente; \
    proporciona un breve resumen de este sitio web en formato Markdown. \
    Si incluye noticias, productos o anuncios, resúmelos también.\n\n"
    user_prompt += website.text
    return user_prompt


# Puedes ver cómo esta función crea exactamente el formato anterior

def messages_for(website):
    return [
        {"role": "user", "content": user_prompt_for(website)}
    ]

##frog = Website("https://jobs.tec.mx/tecnologico-de-monterrey/job/Puebla-Especialista-de-Laboratorio-de-Ingenier%C3%ADa-Industrial/1314202400/")
##messages_for(frog)

def summarize(url):
    website = Website(url)
    response = ollama.chat(
        model=MODEL, 
        messages=messages_for(website)
    )
    return response['message']['content']

## summarize("https://jobs.tec.mx/tecnologico-de-monterrey/job/Puebla-Especialista-de-Laboratorio-de-Ingenier%C3%ADa-Industrial/1314202400/")

# Una función para mostrar esto de forma clara en la salida de Jupyter, usando markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

display_summary("https://jobs.tec.mx/tecnologico-de-monterrey/job/Puebla-Especialista-de-Laboratorio-de-Ingenier%C3%ADa-Industrial/1314202400/")

# 📌 Resumen del puesto: Especialista de Laboratorio de Ingeniería Industrial

---

## 🔍 Visión general

- **Institución**: Tecnológico de Monterrey (Campus Puebla)  
- **Tipo de empleo**: Presencial  
- **Fecha de publicación**: 2 oct. 2025  
- **Categoría**: Ingeniería y Ciencias (Social Media, Marketing)

El Tecnológico de Monterrey busca a un **Especialista de Laboratorio de Ingeniería Industrial** que administre y brinde soporte a los laboratorios de Ingeniería Industrial, Metrología e Ingeniería Automotriz en la **EIC Puebla**.

---

## 🎯 Responsabilidades clave

| Área | Detalle |
|------|---------|
| **Atención a usuarios** | Soporte a alumnos y profesores en actividades experimentales. |
| **Impartición de prácticas y talleres** | Diseño y ejecución de talleres experimentales y asesoría en el uso correcto de equipos. |
| **Mantenimiento** | Planificación y ejecución de mantenimiento preventivo y correctivo. |
| **Inventario y compras** | Levantamiento de inventarios y gestión de insumos necesarios. |
| **Seguridad** | Cumplimiento de reglamentos y aseguramiento de condiciones seguras. |
| **Acreditación** | Provisión de información para procesos de acreditación académica. |
| **Apoyo a investigación** | Asistencia a profesores y estudiantes de licenciatura y posgrado. |
| **Atracción de alumnos** | Diseño de talleres dirigidos a candidatos potenciales. |

---

## 📚 Requisitos

| Categoría | Requisito |
|-----------|-----------|
| **Formación** | Licenciatura en Ingeniería Mecatrónica, Industrial o Mecánica. |
| **Experiencia** | ≥ 2 años en administración/coordinación de laboratorios y enseñanza de prácticas. |
| **Conocimientos técnicos** | Paquetería Microsoft, Tecnologías de Industria 4.0, dominio del inglés. |

---

## 🗺️ Ubicación

- **Ciudad**: Puebla, México  
- **Dirección**: Av. Eugenio Garza Sada 2501 Sur, Col. Tecnológico, CP 64849, Monterrey, N.L. (cerca de campus)

---

## 📢 Notas adicionales

- **Igualdad de oportunidades**: No se discrimina por edad, origen, género, orientación sexual, estado civil, discapacidad, etc.  
- **Redes sociales**: Se invita a seguir a Jobs.tec.mx en Instagram, LinkedIn, Facebook y TikTok.  
- **Proceso de aplicación**: Enviar candidatura a través del portal jobs.tec.mx o con LinkedIn.

---

## 📣 ¿Hay noticias o productos?  

No se incluyen noticias, productos ni anuncios adicionales en la página; la única información relevante es la oferta de empleo y los requisitos de la posición.  

---